This file generates a faux isotropic background that contains the same number of photons in both the true isotropic and anisotropic backgrounds

In [35]:
%reset

In [36]:
import sys
sys.path.append('/home/users/ids29/DGRB')

In [37]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed, parallel_backend
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal
import sources.FermiBackgrounds as FermiBackgrounds
from astropy import units as u


%matplotlib inline

In [38]:
grains=10_000
num_workers = 48

In [39]:
energy_range = [1_000, 100_000] #MeV 
max_radius = 220 #kpc #This is the virial radius of the Milky Way
exposure = 2_000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut_degrees = 180 #degrees # 180 degress is full sky
lat_cut_degrees = 0 #degrees # 0 degrees is full sky
luminosity_range = np.array([1e13, 1e53]) # Minimum value set by considering source at distnace of closest approach by line of sight at 40 degrees and receiving 0.1 photon at detector side. CHANGE THIS FOR FINAL PROBLEM.
                                            # Maximum value set by that value of luminosity that suffieciently suppreses the luminoisity function at the higher end of the luminosity range.


parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
angular_cut = angular_cut_degrees*u.deg.to('rad') # radians
angular_cut_gen =  min(angular_cut*1.5, np.pi) # radians
lat_cut = lat_cut_degrees*u.deg.to('rad') # radians
lat_cut_gen = lat_cut*0.5 # radians

In [40]:
def build_aegis_object():
    engine = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut , lat_cut, flux_cut, energy_range_gen=energy_range_gen, verbose = False)
    engine.angular_cut_gen, engine.lat_cut_gen = angular_cut_gen, lat_cut_gen
    return engine


In [41]:
# 1) Point to your data directory and choose a channel
data_dir = "/home/users/ids29/DGRB/data/dm_spectra/"         # must contain AtProduction_gammas.dat
channel  = "Tau"                           # e.g. 'b' (bottom quarks) or 'Tau'


# 2) Build the DMsignal object
dm = DMsignal.DMsignal(directory=data_dir, channel=channel)

Isotropic Background signal

In [ ]:
# Note: Terminology:
# flux is photon/cm^2/s
# differential flux is photon/cm^2/s/MeV

# Add Fermi background source class
data_root = '/home/users/ids29/DGRB'
fb = FermiBackgrounds.FermiBackgrounds(data_root)

def spectrum_iso_BG(energy, params): # returns spectrum dN/dA/dt/dE/d(solid angle)  for each value of energy specified in the input array 'energy' 
    #such that the area under curve equals the flux (photons/cm^2/s) over the detector's energy range

    dFdEdSr_E_interpfunc = fb.get_isotropic_background_spectrum_func() # returns an interpolation function -> dF/dE/d(solid_angle) versus E
                                
    return dFdEdSr_E_interpfunc(energy) 

# AEGIS functions for Fermi isotropic background
als_iso_BG = [spectrum_iso_BG]

Anisotropic background signal

In [43]:
def make_H_aniso_BG(fb, N_side, roi_mask=None):
    """
    Build H(params) for AEGIS 'healpix_map' using the anisotropic background.

    Returns H(params) -> map_vals, map_E, map_i, N_side, where:
      - map_vals: 2D array shape (N_E, N_pix_included),
                  UNITS: photons / cm^2 / s / sr / MeV   
      - map_E:    1D array energies [MeV]
      - map_i:    1D array of included HEALPix pixel indices (len = N_pix_included)
      - N_side:   int

    Notes
    -----
    * Do NOT multiply by the pixel solid angle; AEGIS applies Ω_pix = 4π / full_map_N_pix.
    * If roi_mask is provided (boolean of length 12*N_side^2), only those pixels are included.
    """

    # Load anisotropic background from FermiBackgrounds.py
    aniso_bg = fb.get_nonistropic_background(N_side=N_side) # returns a dictionary 
    E_grid = aniso_bg["energies_MeV"]                  # (N_E,) # units: MeV
    map_per_sr = aniso_bg["galactic_bg"]                  # (N_E, N_pix)  # units: photon / cm^2 / s / sr / MeV
    N_pix = map_per_sr.shape[1]

    # Sanity checks
    expected = 12 * N_side**2
    if N_pix != expected:
        raise ValueError(f"Anisotropic map has {N_pix} pixels, expected {expected} pixels for N_side={N_side}.")

    # Restrict to ROI if provided
    if roi_mask is None: # full sky
        map_i = np.arange(N_pix, dtype=int)
        map_vals = map_per_sr
    else:
        roi_mask = np.asarray(roi_mask, dtype=bool)
        if roi_mask.size != N_pix:
            raise ValueError(f"roi_mask length ({roi_mask.size}) is not equal to N_pix ({N_pix})")
        map_i = np.arange(N_pix, dtype=int)[roi_mask]  # returns indices of roi_mask which have True entries
        map_vals = map_per_sr[:, map_i]

    
    return map_vals.copy(), E_grid, map_i, N_side

In [44]:
# Note: Terminology:
# flux is photon/cm^2/s
# differential flux is photon/cm^2/s/MeV

# Add Fermi background source class
data_root = '/home/users/ids29/DGRB'
fb = FermiBackgrounds.FermiBackgrounds(data_root)

def H_aniso(params):
    # We don't care about 'params'
    N_side = 64
    map_vals, E_grid, map_i, N_side = make_H_aniso_BG(fb, N_side=N_side, roi_mask=None)
    return map_vals, E_grid, map_i, N_side

# AEGIS functions for Fermi anisotropic background
als_aniso_BG = [H_aniso]

In [45]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(my_AEGIS, params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-3)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/users/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/users/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

Generate Data

In [46]:
# Instatantiate the AEGIS object for Background
obj_AEGIS = build_aegis_object()

# Configure the AEGIS object for Isotropic Background
obj_AEGIS.abun_lum_spec = [als_iso_BG]
obj_AEGIS.source_class_list = ['isotropic_diffuse']

# Run the simulator for Isotropic Background data
photon_data_iso_BG = simulator(obj_AEGIS, torch.tensor([-1])) # Dummy params value

# Configure the AEGIS object for Anisotropic Background
obj_AEGIS.abun_lum_spec =  [als_aniso_BG]
obj_AEGIS.source_class_list = ['healpix_map']

# Run the simulator for Background data
photon_data_aniso_BG = simulator(obj_AEGIS, torch.tensor([-1])) # Dummy params value

# Save the photon information and theta parameters to files.
with open(f'data_iso_BG.pkl', 'wb') as f:
    pickle.dump(photon_data_iso_BG, f)
with open(f'data_aniso_BG.pkl', 'wb') as f:
    pickle.dump(photon_data_aniso_BG, f)

Healpix map 0 has 1376256 pixels


In [47]:
print(f"Total number of photons in the Isotropic background = {photon_data_iso_BG['energies'].size}")
print(f"Total number of photons in the Anisotropic background = {photon_data_aniso_BG['energies'].size}")

Total number of photons in the Isotropic background = 358196
Total number of photons in the Anisotropic background = 6849595
